In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [14]:
# Parâmetro w ajustável
w = 30      #####     HIPERMARÂMTRO       #####

# Definindo o período dos últimos 5 anos
end_date = pd.to_datetime("today").strftime("%Y-%m-%d")
start_date = (pd.to_datetime("today") - pd.DateOffset(years=5)).strftime("%Y-%m-%d")

# Função para calcular o retorno logarítmico
def log_return(series):
    return np.log(series / series.shift(1))

# Baixando os dados históricos do Ethereum, Bitcoin e Solana
eth_data = yf.download("ETH-USD", start=start_date, end=end_date)
btc_data = yf.download("BTC-USD", start=start_date, end=end_date)
sol_data = yf.download("SOL-USD", start=start_date, end=end_date)

# Calculando o retorno logarítmico diário e fazendo o shift para corrigir as datas
eth_data['Return'] = log_return(eth_data['Close']).shift(-2)
btc_data['Return'] = log_return(btc_data['Close']).shift(-2)
sol_data['Return'] = log_return(sol_data['Close']).shift(-2)

# Sincronizando os dados para garantir que todas as moedas tenham os mesmos dias
combined_data = pd.concat([eth_data['Return'], btc_data['Return'], sol_data['Return']], axis=1, keys=['ETH_Return', 'BTC_Return', 'SOL_Return']).dropna()

# Criando o dataframe final
rows = []
for i in range(w, len(combined_data)):
    eth_returns = combined_data['ETH_Return'].iloc[i-w:i].values
    btc_returns = combined_data['BTC_Return'].iloc[i-w:i].values
    sol_returns = combined_data['SOL_Return'].iloc[i-w:i].values
    eth_age = (combined_data.index[i] - eth_data.index[0]).days
    eth_return_today = combined_data['ETH_Return'].iloc[i]
    
    # Concatenando os dados na linha
    row = np.concatenate([eth_returns, btc_returns, sol_returns, [eth_age], [eth_return_today]])
    rows.append(row)

# Nome das colunas
columns = [f"ETH_Return_Day_{j+1}" for j in range(w)] + \
          [f"BTC_Return_Day_{j+1}" for j in range(w)] + \
          [f"SOL_Return_Day_{j+1}" for j in range(w)] + \
          ['ETH_Age', 'ETH_Return_Today']

# Criando o DataFrame final com a data correta como índice
df_final = pd.DataFrame(rows, columns=columns, index=combined_data.index[w:])

df_final.tail()
df_final.shape
df_final

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,ETH_Return_Day_1,ETH_Return_Day_2,ETH_Return_Day_3,ETH_Return_Day_4,ETH_Return_Day_5,ETH_Return_Day_6,ETH_Return_Day_7,ETH_Return_Day_8,ETH_Return_Day_9,ETH_Return_Day_10,...,SOL_Return_Day_23,SOL_Return_Day_24,SOL_Return_Day_25,SOL_Return_Day_26,SOL_Return_Day_27,SOL_Return_Day_28,SOL_Return_Day_29,SOL_Return_Day_30,ETH_Age,ETH_Return_Today
Date,,,,,,,,,,,,,,,,,,,,,
2020-05-10,0.018327,-0.030642,0.008391,-0.027726,0.116098,-0.003018,0.085258,-0.028760,-0.052668,0.002554,...,-0.017883,-0.039192,-0.030456,-0.015550,-0.013229,-0.000653,-0.118385,-0.041365,209.0,0.018121
2020-05-11,-0.030642,0.008391,-0.027726,0.116098,-0.003018,0.085258,-0.028760,-0.052668,0.002554,0.055521,...,-0.039192,-0.030456,-0.015550,-0.013229,-0.000653,-0.118385,-0.041365,0.044232,210.0,0.050877
2020-05-12,0.008391,-0.027726,0.116098,-0.003018,0.085258,-0.028760,-0.052668,0.002554,0.055521,0.013215,...,-0.030456,-0.015550,-0.013229,-0.000653,-0.118385,-0.041365,0.044232,0.021956,211.0,0.018680
2020-05-13,-0.027726,0.116098,-0.003018,0.085258,-0.028760,-0.052668,0.002554,0.055521,0.013215,0.022489,...,-0.015550,-0.013229,-0.000653,-0.118385,-0.041365,0.044232,0.021956,0.033203,212.0,-0.036768
2020-05-14,0.116098,-0.003018,0.085258,-0.028760,-0.052668,0.002554,0.055521,0.013215,0.022489,0.032639,...,-0.013229,-0.000653,-0.118385,-0.041365,0.044232,0.021956,0.033203,-0.016594,213.0,0.025510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-07,0.026287,0.013100,-0.021035,0.009333,0.033240,-0.009469,-0.041233,-0.011097,0.020024,0.011895,...,-0.049210,-0.034338,-0.025981,0.045637,-0.003222,0.025805,-0.016876,-0.004157,1820.0,-0.029767
2024-10-08,0.013100,-0.021035,0.009333,0.033240,-0.009469,-0.041233,-0.011097,0.020024,0.011895,0.039316,...,-0.034338,-0.025981,0.045637,-0.003222,0.025805,-0.016876,-0.004157,-0.028858,1821.0,0.006555
2024-10-09,-0.021035,0.009333,0.033240,-0.009469,-0.041233,-0.011097,0.020024,0.011895,0.039316,0.038335,...,-0.025981,0.045637,-0.003222,0.025805,-0.016876,-0.004157,-0.028858,-0.003071,1822.0,0.021848


In [3]:
# Criando a função First Transformation
def FirstTransform(df, W):
    vet = []
    Linhas, Colunas = df.shape
    for i in range(Linhas, W+1, -1):
        vet.append(df.iloc[(i-W-1):i])
    return vet

df = df_final
W = 300      ####     HIPER PARAMETRO     #####
vetor  = FirstTransform(df, W)
len(vetor)
vetor[0]


,ETH_Return_Day_1,ETH_Return_Day_2,ETH_Return_Day_3,ETH_Return_Day_4,ETH_Return_Day_5,ETH_Return_Day_6,ETH_Return_Day_7,ETH_Return_Day_8,ETH_Return_Day_9,ETH_Return_Day_10,...,SOL_Return_Day_23,SOL_Return_Day_24,SOL_Return_Day_25,SOL_Return_Day_26,SOL_Return_Day_27,SOL_Return_Day_28,SOL_Return_Day_29,SOL_Return_Day_30,ETH_Age,ETH_Return_Today
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-16,0.001021,0.025109,0.004478,-0.043031,0.063677,-0.001073,0.009143,0.001566,-0.010187,-0.017537,...,0.033321,-0.059832,-0.026299,0.031976,0.066321,-0.043947,0.015147,-0.034238,1524.0,0.009422
2023-12-17,0.025109,0.004478,-0.043031,0.063677,-0.001073,0.009143,0.001566,-0.010187,-0.017537,0.010754,...,-0.059832,-0.026299,0.031976,0.066321,-0.043947,0.015147,-0.034238,0.046058,1525.0,-0.017930
2023-12-18,0.004478,-0.043031,0.063677,-0.001073,0.009143,0.001566,-0.010187,-0.017537,0.010754,-0.009516,...,-0.026299,0.031976,0.066321,-0.043947,0.015147,-0.034238,0.046058,-0.018426,1526.0,0.010977
2023-12-19,-0.043031,0.063677,-0.001073,0.009143,0.001566,-0.010187,-0.017537,0.010754,-0.009516,0.011085,...,0.031976,0.066321,-0.043947,0.015147,-0.034238,0.046058,-0.018426,0.118729,1527.0,0.016946
2023-12-20,0.063677,-0.001073,0.009143,0.001566,-0.010187,-0.017537,0.010754,-0.009516,0.011085,0.016709,...,0.066321,-0.043947,0.015147,-0.034238,0.046058,-0.018426,0.118729,0.132700,1528.0,0.038104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-07,0.026287,0.013100,-0.021035,0.009333,0.033240,-0.009469,-0.041233,-0.011097,0.020024,0.011895,...,-0.049210,-0.034338,-0.025981,0.045637,-0.003222,0.025805,-0.016876,-0.004157,1820.0,-0.029767
2024-10-08,0.013100,-0.021035,0.009333,0.033240,-0.009469,-0.041233,-0.011097,0.020024,0.011895,0.039316,...,-0.034338,-0.025981,0.045637,-0.003222,0.025805,-0.016876,-0.004157,-0.028858,1821.0,0.006555
2024-10-09,-0.021035,0.009333,0.033240,-0.009469,-0.041233,-0.011097,0.020024,0.011895,0.039316,0.038335,...,-0.025981,0.045637,-0.003222,0.025805,-0.016876,-0.004157,-0.028858,-0.003071,1822.0,0.021848


In [4]:
def Bagging(df, n, gamma):
    df_bagged = []
    for i in range(gamma):
        aux = df.sample(n = n, random_state = i)
        aux = aux.sort_index()
        df_bagged.append(aux)
    return df_bagged

In [5]:
vetor_alvos = []
for i in range(len(vetor)):
    vetor_alvos.append((pd.DataFrame(vetor[i].iloc[-1])).T)
    vetor[i] = vetor[i].drop(vetor[i].index[-1])

vetor[0]

,ETH_Return_Day_1,ETH_Return_Day_2,ETH_Return_Day_3,ETH_Return_Day_4,ETH_Return_Day_5,ETH_Return_Day_6,ETH_Return_Day_7,ETH_Return_Day_8,ETH_Return_Day_9,ETH_Return_Day_10,...,SOL_Return_Day_23,SOL_Return_Day_24,SOL_Return_Day_25,SOL_Return_Day_26,SOL_Return_Day_27,SOL_Return_Day_28,SOL_Return_Day_29,SOL_Return_Day_30,ETH_Age,ETH_Return_Today
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-16,0.001021,0.025109,0.004478,-0.043031,0.063677,-0.001073,0.009143,0.001566,-0.010187,-0.017537,...,0.033321,-0.059832,-0.026299,0.031976,0.066321,-0.043947,0.015147,-0.034238,1524.0,0.009422
2023-12-17,0.025109,0.004478,-0.043031,0.063677,-0.001073,0.009143,0.001566,-0.010187,-0.017537,0.010754,...,-0.059832,-0.026299,0.031976,0.066321,-0.043947,0.015147,-0.034238,0.046058,1525.0,-0.017930
2023-12-18,0.004478,-0.043031,0.063677,-0.001073,0.009143,0.001566,-0.010187,-0.017537,0.010754,-0.009516,...,-0.026299,0.031976,0.066321,-0.043947,0.015147,-0.034238,0.046058,-0.018426,1526.0,0.010977
2023-12-19,-0.043031,0.063677,-0.001073,0.009143,0.001566,-0.010187,-0.017537,0.010754,-0.009516,0.011085,...,0.031976,0.066321,-0.043947,0.015147,-0.034238,0.046058,-0.018426,0.118729,1527.0,0.016946
2023-12-20,0.063677,-0.001073,0.009143,0.001566,-0.010187,-0.017537,0.010754,-0.009516,0.011085,0.016709,...,0.066321,-0.043947,0.015147,-0.034238,0.046058,-0.018426,0.118729,0.132700,1528.0,0.038104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-06,0.010144,0.026287,0.013100,-0.021035,0.009333,0.033240,-0.009469,-0.041233,-0.011097,0.020024,...,-0.038628,-0.049210,-0.034338,-0.025981,0.045637,-0.003222,0.025805,-0.016876,1819.0,0.007423
2024-10-07,0.026287,0.013100,-0.021035,0.009333,0.033240,-0.009469,-0.041233,-0.011097,0.020024,0.011895,...,-0.049210,-0.034338,-0.025981,0.045637,-0.003222,0.025805,-0.016876,-0.004157,1820.0,-0.029767
2024-10-08,0.013100,-0.021035,0.009333,0.033240,-0.009469,-0.041233,-0.011097,0.020024,0.011895,0.039316,...,-0.034338,-0.025981,0.045637,-0.003222,0.025805,-0.016876,-0.004157,-0.028858,1821.0,0.006555


In [6]:
def Model(df):

    df = df[['ETH_Return_Day_30', 'ETH_Return_Today']].copy()
    X = df.drop(columns=['ETH_Return_Today']).values
    y = df['ETH_Return_Today'].values
    
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))
    X_array = np.array(X_scaled)
    y_array = np.array(y_scaled)

    # Crie as sequências com o número de timesteps
    X_lstm = []
    y_lstm = []
    timesteps = 30
    for i in range(timesteps, len(X_array)):
        X_lstm.append(X_array[i-timesteps+1:i+1, :])  # Pega 'timesteps' linhas anteriores
        y_lstm.append(y_scaled[i])  # Alvo é o valor do dia seguinte

    X_lstm = np.array(X_lstm)
    y_lstm = np.array(y_lstm)

    # Separar o X_train (todos exceto o último elemento)
    X_train = X_lstm[:-1]
    y_train = y_lstm[:-1]

    # X_test e y_test
    X_test = X_lstm[-1:]
    y_test = y_lstm[-1:]

    # Defina o modelo
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2]))) # [samples, time steps, features]
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Treine o modelo
    model.fit(X_train, y_train, epochs=30, batch_size=64, verbose = 0)

    y_hat_scaled = model.predict(X_test)
    y_hat = scaler_y.inverse_transform(y_hat_scaled)[0][0]

    y_real = scaler_y.inverse_transform(y_test)[0][0]

    return y_hat

In [13]:
import math

n = 270       #####       HIPERPARÂMETRO      #####
gamma = 1      #####       HIPERPARÂMETRO      #####

ganho = 1
for i in range(len(vetor) - 30, len(vetor)):
    X_inicial = vetor[i].copy() # X_inicial eh uma matriz de W linhas e C+1 colunas
    X_bagged = Bagging(X_inicial, n, gamma) # X_bagged vira um vetor em que cada elemento eh uma coluna de n linhas e C+1 colunas, que são os baggeds de um elemento do vetor VETOR
    sum_y_hat = 0
    for j in range(len(X_bagged)):
        df = pd.concat([X_bagged[j], vetor_alvos[i]], ignore_index=True)
        y_hat = Model(df)
        sum_y_hat += y_hat
        print("Terminei um elemento do bagging \n")
    print("______________________________________ \n")  
    print("Terminei o dia ", i, "\n\n")        
    y_hat = sum_y_hat / len(X_bagged)   # y_hat é o retorno logaritmo previsto

    exp_return = math.exp(y_hat) - 1    # é o nosso retorno percentual esperado

    if(exp_return > 0):
        y_real = vetor_alvos[i].iloc[-1, -1]
        y_real_perc = math.exp(y_real) - 1
        ganho *= (1 + y_real_perc)
    print("Ganho atual:", ganho, "\n")
print(ganho)

C:\Users\delve\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
Terminei um elemento do bagging 



C:\Users\delve\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
Terminei um elemento do bagging 



C:\Users\delve\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
Terminei um elemento do bagging 



C:\Users\delve\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
Terminei um elemento do bagging 



C:\Users\delve\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
Terminei um elemento do bagging 

______________________________________ 

Terminei o dia  1314 


Ganho atual: 1.0506298300713457 

1.0506298300713457
